In [ ]:
#Importing libraries

In [ ]:
import pandas as pd
import sqlite3
from flask import Flask, request
import csv

In [ ]:
#A SQLite Database

In [ ]:
conn = sqlite3.connect('crypto_trades.db')

In [ ]:
#Trades Table

In [ ]:
conn.execute('''
    CREATE TABLE IF NOT EXISTS Trades (
        Trade_ID INTEGER PRIMARY KEY AUTOINCREMENT,
        UTC_Time TEXT,
        Operation TEXT,
        Base_Coin TEXT,
        Quote_Coin TEXT,
        Amount REAL,
        Price REAL
    )
''')

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'

    file = request.files['/content/KoinX Assignment CSV Sample.csv']

    if file.filename == '':
      return 'No selected file'

    if file:
        trade_data = []
        file.seek(0)  # Reset file pointer to the beginning
        csv_reader = csv.DictReader(file)

        for row in csv_reader:
            trade_data.append((
                row['UTC_Time'],
                row['Operation'],
                row['Market'].split('/')[0],  # Base coin
                row['Market'].split('/')[1],  # Quote coin
                row['Buy/Sell Amount'],
                row['Price']
            ))


In [ ]:
conn = sqlite3.connect('trade_data.db')

c1 = conn.cursor()
c1.execute('''
         CREATE TABLE IF NOT EXISTS trade_data(utc_time TEXT, operation TEXT, base_coin TEXT, quote_coin TEXT, amount REAL, price REAL) ''')
c1.executemany("INSERT INTO trades (UTC_Time, Operation, Market, Buy_Sell_Amount, PRICE) VALUES (?, ?, ?, ?, ?)", trade_data)
conn.commit()
conn.close()
return 'File uploaded successfully'

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:

def upload_data():
    conn = sqlite3.connect('trade_data.db')
    c1 = conn.cursor()

    c1.execute('''
        CREATE TABLE IF NOT EXISTS trade_data(
            utc_time TEXT,
            operation TEXT,
            base_coin TEXT,
            quote_coin TEXT,
            amount REAL,
            price REAL
        )
    ''')

    trade_data = [
        ('26-09-2022 11:21:02', 'Buy', 'BTC', 'INR', 25, 1000),
        ('27-09-2022 11:21:02', 'Sell', 'BTC', 'INR', 10, 2000),
        ('28-09-2022 11:21:02', 'Buy', 'MATIC', 'INR', 100, 12),

    ]

    c1.executemany('''
        INSERT INTO trade_data (utc_time, operation, base_coin, quote_coin, amount, price)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', trade_data)

    conn.commit()
    conn.close()

    return 'File uploaded successfully'

upload_data()

'File uploaded successfully'

In [ ]:
@app.route('/balance', methods=['GET'])
def get_bal():
    timestamp = request.args.get('timestamp')

    if not timestamp:
        return 'Missing timestamp parameter', 400

    conn = sqlite3.connect('trades.db')
    c = conn.cursor()

    c.execute('''
        SELECT base_coin, SUM(CASE WHEN operation = 'buy' THEN amount ELSE -amount END) as balance
        FROM trades
        WHERE UTC_Time < ?
        GROUP BY base_coin
    ''', (timestamp,))

    balances = {row[0]: row[1] for row in c.fetchall()}
    conn.close()

    return jsonify(balances)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
